# Create album

In [34]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
import os 
import pyreadr
import numpy as np
import pyreadr

In [35]:
DF = pyreadr.read_r('/Users/mxenoc/Desktop/workspace/butterfly/data/omics.RData')
DF = DF["DF"]

In [36]:
import butterfly.album

In [37]:
# Choose your responses, predictors, and size for the image
predictor = 'plasma_s'
response = 'plasma_l'
pixels = 40

In [38]:
album = butterfly.album.create_album(DF, predictor, pixels)

In [39]:
import butterfly.CNN

In [40]:
#Decide which dataset you want to predict
response = [col for col in DF if col.startswith(response)]
response.append("patientID")
response.append("trimester")    
response_df = DF[response]
#Make sure your B trimesters are properly converted to 4

In [41]:
#Get every combination of patient IDs in pairs of m's
# Function which returns subset or r length from n 
from itertools import combinations 

m = 2
IDs = response_df['patientID'].unique()
IDs = IDs.tolist()
exclude = list(combinations(IDs, m))

In [ ]:
#from joblib import Parallel, delayed
#import multiprocessing

#num_cores = multiprocessing.cpu_count()

#results = Parallel(n_jobs=num_cores)(delayed(processInput)(i) for i in inputs)

In [27]:
XX = album
yy = response_df['plasma_l1'].values

In [29]:
cv = 0

In [33]:
#for cv in range(len(exclude)):
#Create your calibration and validation datasets
pt1ex = response_df.index[response_df['patientID'] == exclude[cv][0]].tolist()
pt2ex = response_df.index[response_df['patientID'] == exclude[cv][1]].tolist()
ptex = pt1ex+pt2ex

#Divide in calibration and validation
X_c = np.delete(XX, ptex, 0)
y_c = np.delete(yy, ptex, 0)

X_v = [XX[i]  for i in ptex]
y_v = [yy[i] for i in ptex]

#Create your CNN
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=2, activation='relu', 
                 input_shape=(pixels, pixels)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# fit model
model.fit(X_c, y_c, epochs=1000, verbose=0)

NameError: name 'Sequential' is not defined

In [ ]:
# demonstrate prediction
x_input = X_v

y_pred = model.predict(x_input, verbose=0)